In [1]:
import os
import hdf5storage
import mat73
import pandas as pd
import numpy as np

In [2]:
data_dir = "datasets/autism data/WT-M355/"
fname = os.path.join(data_dir, "all_pez_data.mat")
mat = mat73.loadmat(fname)
real_keys = [k for k in mat.keys() if not (k.startswith("__") and k.endswith("__"))]
td_name = real_keys[0]
df = pd.DataFrame(mat[td_name])

In [3]:
# Keep only rows with data: (get rid of rows full of None)
df_new = df[df['success'] <= 1]
# Re-indexing to consecutive indices
df_new.index = range(len(df_new))
df_new["success"]

0      0.0
1      1.0
2      0.0
3      1.0
4      1.0
      ... 
864    1.0
865    1.0
866    1.0
867    0.0
868    0.0
Name: success, Length: 869, dtype: object

In [4]:
df_new.shape

(869, 26)

In [5]:
# padd the velocity and acceleration to have same size (same number of timesteps) as trajectory (hand in our dataset):
# padd with zeros at the end because we consider that the hand is back on perch-- end of movement; also we know that velocity at timestep t is calculated wrt to positions at (t+1) and t, same for the acc; so end of the arrays vel and acc need to be padded
i = 0

for no_trial in range(len(df_new)):
    try:
        if df_new.vel[no_trial].shape[0] < df_new.hand[no_trial].shape[1]:
            length_diff = df_new.hand[no_trial].shape[1] - df_new.vel[no_trial].shape[0]
            df_new.vel[no_trial] = np.pad(df_new.vel[no_trial], (0, length_diff), mode='constant', constant_values=0)
        elif df_new.vel[no_trial].shape[0] > df_new.hand[no_trial].shape[1]:
            df_new.vel[no_trial] = df_new.vel[no_trial][:len(df_new.hand[no_trial])]

        if df_new.acc[no_trial].shape[0] < df_new.hand[no_trial].shape[1]:
            length_diff = df_new.hand[no_trial].shape[1] - df_new.acc[no_trial].shape[0]
            df_new.acc[no_trial] = np.pad(df_new.acc[no_trial], (0, length_diff), mode='constant', constant_values=0)
        elif df_new.acc[no_trial].shape[0] > df_new.hand[no_trial].shape[1]:
            df_new.acc[no_trial] = df_new.acc[no_trial][:len(df_new.hand[no_trial])]


    except:
        print("exception")
        i +=1
        print(df_new.vel[no_trial].shape)
        df_new = df_new.drop(no_trial)
        # Re-indexing to consecutive indices
        df_new.index = range(len(df_new))


# TO DO: LOOK INTO HOW TO PAD OR REMOVE VALUES and also transpose!!!!!! FROM OTHER FIELDS SUCH AS digit1 etc TO MAKE EVERYTHING THE SAME SIZE AS THE hand FIELD.


C:\Users\MARA\AppData\Local\Temp\ipykernel_14944\2801494275.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.vel[no_trial] = np.pad(df_new.vel[no_trial], (0, length_diff), mode='constant', constant_values=0)
C:\Users\MARA\AppData\Local\Temp\ipykernel_14944\2801494275.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.acc[no_trial] = np.pad(df_new.acc[no_trial], (0, length_diff), mode='constant', constant_values=0)


In [6]:
# Transpose the hand trajectory to be timesteps x dimensions, because when searching for time-varying fields in pyaldata for e.g. in combine_time_bins, we account for shape[0]

for trial_no in range(len(df_new.hand)):
    df_new.hand[trial_no] = np.transpose(df_new.hand[trial_no])

# TO DO: TRANSPOSE OTHER FIELDS AS WELL, E.G. DIGIT1?

C:\Users\MARA\AppData\Local\Temp\ipykernel_14944\3963034535.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.hand[trial_no] = np.transpose(df_new.hand[trial_no])


In [7]:
df_new.hand[77].shape

(2498, 3)

In [8]:
df_new.shape

(869, 26)

In [9]:
if 'm1' in df_new.columns:
    df_new.rename(columns={'m1':'m1_spikes'}, inplace=True)

    # Cut intertrial recordings from neural data, ie only keep the neural data for which we have recorded trajectories:
    try:
        for no_trial in range(len(df_new)):
            df_new.m1_spikes[no_trial] = np.delete(df_new.m1_spikes[no_trial], list(range(2500))+list(range(2500+df_new.hand[no_trial].shape[0], df_new.m1_spikes[no_trial].shape[1])), 1)
    except:
        print("exception")
        df_new = df_new.drop(no_trial)
        # Re-indexing to consecutive indices
        df_new.index = range(len(df_new))
        
        

if 'dcn' in df_new.columns:
    df_new.rename(columns={'dcn':'dcn_spikes'}, inplace=True)

        # Cut intertrial recordings from neural data, ie only keep the neural data for which we have recorded trajectories:
    try:
        for no_trial in range(len(df_new)):
            df_new.dcn_spikes[no_trial] = np.delete(df_new.dcn_spikes[no_trial], list(range(2500))+list(range(2500+df_new.hand[no_trial].shape[0], df_new.dcn_spikes[no_trial].shape[1])), 1)
    except:
        print("exception")
        df_new = df_new.drop(no_trial)
        # Re-indexing to consecutive indices
        df_new.index = range(len(df_new))


if 'str' in df_new.columns:
    df_new.rename(columns={'str':'str_spikes'}, inplace=True)

        # Cut intertrial recordings from neural data, ie only keep the neural data for which we have recorded trajectories:
    try:
        for no_trial in range(len(df_new)):
            df_new.str_spikes[no_trial] = np.delete(df_new.str_spikes[no_trial], list(range(2500))+list(range(2500+df_new.hand[no_trial].shape[0], df_new.str_spikes[no_trial].shape[1])), 1)
    except:
        print("exception")
        df_new = df_new.drop(no_trial)
        # Re-indexing to consecutive indices
        df_new.index = range(len(df_new))
    


if 'thal' in df_new.columns:
    df_new.rename(columns={'thal':'thal_spikes'}, inplace=True)

        # Cut intertrial recordings from neural data, ie only keep the neural data for which we have recorded trajectories:
    try:
        for no_trial in range(len(df_new)):
            df_new.thal_spikes[no_trial] = np.delete(df_new.thal_spikes[no_trial], list(range(2500))+list(range(2500+df_new.hand[no_trial].shape[0], df_new.thal_spikes[no_trial].shape[1])), 1)
    except:
        print("exception")
        df_new = df_new.drop(no_trial)
        # Re-indexing to consecutive indices
        df_new.index = range(len(df_new))
        

C:\Users\MARA\AppData\Local\Temp\ipykernel_14944\1567369540.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.rename(columns={'m1':'m1_spikes'}, inplace=True)
C:\Users\MARA\AppData\Local\Temp\ipykernel_14944\1567369540.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.m1_spikes[no_trial] = np.delete(df_new.m1_spikes[no_trial], list(range(2500))+list(range(2500+df_new.hand[no_trial].shape[0], df_new.m1_spikes[no_trial].shape[1])), 1)


C:\Users\MARA\AppData\Local\Temp\ipykernel_14944\1567369540.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.rename(columns={'dcn':'dcn_spikes'}, inplace=True)
C:\Users\MARA\AppData\Local\Temp\ipykernel_14944\1567369540.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.dcn_spikes[no_trial] = np.delete(df_new.dcn_spikes[no_trial], list(range(2500))+list(range(2500+df_new.hand[no_trial].shape[0], df_new.dcn_spikes[no_trial].shape[1])), 1)
C:\Users\MARA\AppData\Local\Temp\ipykernel_14944\1567369540.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

exception


In [10]:
df_new.shape

(868, 26)

In [11]:
df_new.str_spikes[0].shape

(34, 2498)

In [12]:
df_new.m1_spikes[0].shape

(50, 2498)

In [13]:
df_new["bin_size"] = 0.002

In [14]:
# Replace None with numpy.ndarray with element -2.: (to avoid errors later and allow preprocessing in pyaldata that is mostly performed on numpy.ndarray)

a = None
b = type(None)

for col in df_new.columns:
    for i in range(len(df_new[col])):
        if type(df_new[col][i]) == b:
            df_new[col][i] = np.asarray([-2600.])

C:\Users\MARA\AppData\Local\Temp\ipykernel_14944\2428299202.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[col][i] = np.asarray([-2600.])


In [15]:
# Rename time index fields to contain idx:
time_index_fields = [time_index for time_index in ["mo", "mend", "max_vel", "max_acc", "min_acc", "max_ga", "min_pd", "grab", "sup"] if time_index in df_new.columns]
for time_index in time_index_fields:
    df_new.rename(columns={time_index:"idx_"+time_index}, inplace=True)

df_new.columns

Index(['acc', 'acc_nosmooth', 'da', 'da_nosmooth', 'dcn_spikes', 'digit1',
       'digit3', 'ga', 'ga_nosmooth', 'idx_grab', 'hand', 'm1_spikes',
       'idx_max_ga', 'idx_max_vel', 'idx_mend', 'idx_min_pd', 'idx_mo', 'pd',
       'pd_nosmooth', 'session', 'str_spikes', 'success', 'idx_sup',
       'thal_spikes', 'vel', 'vel_nosmooth', 'bin_size'],
      dtype='object')

In [16]:
# Transpose the spikes cells:

spike_fields = [col for col in df_new.columns if col.endswith("spikes")]
for col in spike_fields:
    for idx in range(len(df_new[col])):
        df_new[col][idx] = np.transpose(df_new[col][idx])

C:\Users\MARA\AppData\Local\Temp\ipykernel_14944\1845384996.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[col][idx] = np.transpose(df_new[col][idx])


In [17]:
# Split in sessions and save a file for each session, with all sessions of an animal in a folder:

data_dir = "datasets/autism data/WT-M355/WT-M355-pyaldata2"
df_session_list = []

for session in range (1, int(max(df_new["session"]))+1):
    df_session_list.append(df_new[df_new["session"] == session])
    hdf5storage.savemat(os.path.join(data_dir, f"WT-M355_session{session}.mat"), {"trial_class": {name: col.values for name, col in df_session_list[session-1].items()}}, format="7.3")